In [1]:
import json
import numpy as np
import re
from grl.p2sro.payoff_table import PayoffTable

# psro_payoff_table_checkpoint_path = "/home/jb/git/grl/grl/data/simple_push_psro/manager_08.41.48PM_Dec-28-2020/payoff_table_checkpoints/payoff_table_checkpoint_25.json"
# psro_policy_nums_path = "/home/jb/git/grl/grl/data/simple_push_psro/manager_08.41.48PM_Dec-28-2020/payoff_table_checkpoints/policy_nums_checkpoint_25.json"

# psro_payoff_table_checkpoint_path = "/home/jb/git/grl/grl/data/simple_push_psro/manager_10.59.03PM_Dec-28-2020/payoff_table_checkpoints/payoff_table_checkpoint_30.json"
# psro_policy_nums_path = "/home/jb/git/grl/grl/data/simple_push_psro/manager_10.59.03PM_Dec-28-2020/payoff_table_checkpoints/policy_nums_checkpoint_30.json"

# psro_payoff_table_checkpoint_path = "/home/jb/git/grl/grl/data/kuhn_psro/manager_06.02.17PM_Jan-14-2021/payoff_table_checkpoints/payoff_table_checkpoint_19.json"
# psro_policy_nums_path = "/home/jb/git/grl/grl/data/kuhn_psro/manager_06.02.17PM_Jan-14-2021/payoff_table_checkpoints/policy_nums_checkpoint_19.json"

psro_payoff_table_checkpoint_path = "/home/jb/git/grl/grl/data/20x_dummy_leduc_psro_dqn_gpu/manager_02.21.29PM_Jan-21-2021/payoff_table_checkpoints/payoff_table_checkpoint_latest.json"
psro_policy_nums_path = "/home/jb/git/grl/grl/data/20x_dummy_leduc_psro_dqn_gpu/manager_02.21.29PM_Jan-21-2021/payoff_table_checkpoints/policy_nums_checkpoint_latest.json"


original_date = re.search("\d\d\.\d\d.\d\d.\w\w\w\w\w\W\d\d\W\d\d\d\d", psro_payoff_table_checkpoint_path)[0]
poker_version = "leduc_poker"

with open(psro_policy_nums_path, "r") as policy_nums_file:
    policy_nums = json.load(policy_nums_file)
largest_fixed_policy_nums = (max(policy_nums['0']['fixed_policies']), max(policy_nums['1']['fixed_policies']))

payoff_table = PayoffTable.from_json_file(json_file_path=psro_payoff_table_checkpoint_path)

In [2]:
from grl.rl_apps.kuhn_poker_p2sro.poker_utils import get_stats_for_single_payoff_table
from grl.rl_apps.kuhn_poker_p2sro.poker_multi_agent_env import PokerMultiAgentEnv
from grl.rl_apps.kuhn_poker_p2sro.config import psro_20x_dummy_leduc_params_gpu
from ray.rllib.agents.dqn import SIMPLE_Q_DEFAULT_CONFIG
from ray.rllib.utils import merge_dicts
from grl.rllib_tools.modified_policies import SimpleQTorchPolicyPatched


env_config = {'version': poker_version, "fixed_players": True, "append_valid_actions_mask_to_obs": True,
                          "dummy_action_multiplier": 20,
}
env = PokerMultiAgentEnv(env_config=env_config)
num_players = 2

stats = get_stats_for_single_payoff_table(
    payoff_table=payoff_table,
    highest_policy_num=min(largest_fixed_policy_nums),
    # highest_policy_num=6,
    poker_env_config=env_config,
    policy_class=SimpleQTorchPolicyPatched,
    policy_config=merge_dicts(SIMPLE_Q_DEFAULT_CONFIG, psro_20x_dummy_leduc_params_gpu(action_space=env.action_space))
)




Failed to import TensorFlow. Please note that TensorFlow is not installed by default when you install TensorFlow Probability. This is so that users can decide whether to install the GPU-enabled TensorFlow package. To use TensorFlow Probability, please install the most recent version of TensorFlow, by following instructions at https://tensorflow.org/install.




Failed to import TensorFlow. Please note that TensorFlow is not installed by default when you install TensorFlow Probability. This is so that users can decide whether to install the GPU-enabled TensorFlow package. To use TensorFlow Probability, please install the most recent version of TensorFlow, by following instructions at https://tensorflow.org/install.




Failed to import TensorFlow. Please note that TensorFlow is not installed by default when you install TensorFlow Probability. This is so that users can decide whether to install the GPU-enabled TensorFlow package. To use TensorFlow Probability, please install the most r

2021-01-21 20:33:36,087	INFO services.py:1092 -- View the Ray dashboard at http://127.0.0.1:8266
2021-01-21 20:33:36,616	INFO catalog.py:306 -- Wrapping <class 'grl.rllib_tools.valid_actions_fcnet.get_valid_action_fcn_class.<locals>.ValidActionFullyConnectedNetwork'> as None
2021-01-21 20:33:37,575	INFO catalog.py:306 -- Wrapping <class 'grl.rllib_tools.valid_actions_fcnet.get_valid_action_fcn_class.<locals>.ValidActionFullyConnectedNetwork'> as None
2021-01-21 20:33:37,579	INFO torch_policy.py:107 -- TorchPolicy running on GPU.
2021-01-21 20:33:37,581	INFO catalog.py:306 -- Wrapping <class 'grl.rllib_tools.valid_actions_fcnet.get_valid_action_fcn_class.<locals>.ValidActionFullyConnectedNetwork'> as None
2021-01-21 20:33:37,583	INFO catalog.py:306 -- Wrapping <class 'grl.rllib_tools.valid_actions_fcnet.get_valid_action_fcn_class.<locals>.ValidActionFullyConnectedNetwork'> as None
2021-01-21 20:33:37,589	INFO torch_policy.py:107 -- TorchPolicy running on GPU.


In [4]:
stats

{'num_policies': [98],
 'exploitability': [0.40914872320107104],
 'timesteps': [643968],
 'episodes': [181672]}

In [3]:

import itertools
import pandas as pd
import plotly.express as px
import plotly.io as pio
# pio.renderers.default = 'png'
pio.renderers.default = "browser"

df = pd.DataFrame.from_dict(data=stats)
fig = px.line(df, x="episodes", y="exploitability",
              title='Leduc Poker PSRO Exploitability',
              # range_x=[0, int(3e6)],
              )
fig.show()

In [ ]:
save_path = f"/home/jb/git/grl/grl/data/psro_stats_{poker_version}_{original_date}.json"
with open(save_path, "+w") as json_file:
    json.dump(stats, json_file)
print(f"saved to {save_path}")